Как аппроксимировать x*y c помощью нейросетки?


почему релу так хорошо работает? она линейна, многое в природе линейно 
→ сравнить графки с сигмоидом

→ глянуть GELU

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define the function
def f(x, y):
    return x * y

# Generate a grid of values
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
x, y = np.meshgrid(x, y)
z = f(x, y)

# Create the plot
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Update layout for better visualization
fig.update_layout(title='3D Plot of f(x, y) = x * y',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  width=800, height=800)

# Show the plot
fig.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Generate the data
x = np.linspace(-10, 10, 100)
y = np.linspace(-10, 10, 100)
x, y = np.meshgrid(x, y)
z = x * y

# Convert data to torch tensors
x = torch.tensor(x.flatten(), dtype=torch.float32).view(-1, 1)
y = torch.tensor(y.flatten(), dtype=torch.float32).view(-1, 1)
z = torch.tensor(z.flatten(), dtype=torch.float32).view(-1, 1)

# Concatenate x and y to create input features
inputs = torch.cat((x, y), dim=1)
outputs = z

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the network
epochs = 2
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs_pred = net(inputs)
    loss = criterion(outputs_pred, outputs)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Plot the results
with torch.no_grad():
    z_pred = net(inputs).numpy().reshape(100, 100)

plt.figure(figsize=(10, 8))
plt.contourf(x.view(100, 100), y.view(100, 100), z_pred, levels=50, cmap='viridis')
plt.colorbar()
plt.title('Neural Network Approximation of f(x, y) = x * y')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim

def f(x, y):
    return x * y

# Generate a grid of values for plotting
x_plot = np.linspace(-1000, 1000, 1000)
y_plot = np.linspace(-1000, 1000, 1000)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
z_plot = f(x_plot, y_plot)

# Generate a grid of values for training
x_train = np.linspace(-10, 10, 100)
y_train = np.linspace(-10, 10, 100)
x_train, y_train = np.meshgrid(x_train, y_train)
z_train = f(x_train, y_train)

# Prepare data for training
X_train = np.stack((x_train.flatten(), y_train.flatten()), axis=-1)
y_train_true = z_train.flatten()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train_true, dtype=torch.float32).unsqueeze(1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate predictions for plotting
X_plot = np.stack((x_plot.flatten(), y_plot.flatten()), axis=-1)
X_plot_tensor = torch.tensor(X_plot, dtype=torch.float32)
z_pred = model(X_plot_tensor).detach().numpy().reshape(*z_plot.shape)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

surface1 = go.Surface(z=z_plot, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface1, row=1, col=1)

surface2 = go.Surface(z=z_pred, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface2, row=1, col=2)

fig.update_layout(title='3D Plot of f(x, y) = x * y and Neural Network Approximation',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  scene2=dict(xaxis_title='x',
                              yaxis_title='y',
                              zaxis_title='z_pred'),
                  width=1600, height=800)

fig.show()


In [ ]:
list(model.fc1.parameters())[0].shape

In [ ]:
X_tensor.shape

In [ ]:
list(model.fc1.parameters())[0]

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim

def f(x, y):
    return x * y

# Generate a grid of values for plotting
x_plot = np.linspace(-1000, 1000, 1000)
y_plot = np.linspace(-1000, 1000, 1000)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
z_plot = f(x_plot, y_plot)

# Generate a grid of values for training
x_train = np.linspace(-10, 10, 100)
y_train = np.linspace(-10, 10, 100)
x_train, y_train = np.meshgrid(x_train, y_train)
z_train = f(x_train, y_train)

# Prepare data for training
X_train = np.stack((x_train.flatten(), y_train.flatten()), axis=-1)
y_train_true = z_train.flatten()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train_true, dtype=torch.float32).unsqueeze(1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate predictions for plotting
X_plot = np.stack((x_plot.flatten(), y_plot.flatten()), axis=-1)
X_plot_tensor = torch.tensor(X_plot, dtype=torch.float32)
z_pred = model(X_plot_tensor).detach().numpy().reshape(*z_plot.shape)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

surface1 = go.Surface(z=z_plot, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface1, row=1, col=1)

surface2 = go.Surface(z=z_pred, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface2, row=1, col=2)

fig.update_layout(title='3D Plot of f(x, y) = x * y and Neural Network Approximation',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  scene2=dict(xaxis_title='x',
                              yaxis_title='y',
                              zaxis_title='z_pred'),
                  width=1600, height=800)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim

def f(x, y):
    return x * y

# Generate a grid of values for plotting
x_plot = np.linspace(-1000, 1000, 1000)
y_plot = np.linspace(-1000, 1000, 1000)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
z_plot = f(x_plot, y_plot)

# Generate a grid of values for training
x_train = np.linspace(-10, 10, 100)
y_train = np.linspace(-10, 10, 100)
x_train, y_train = np.meshgrid(x_train, y_train)
z_train = f(x_train, y_train)

# Prepare data for training
X_train = np.stack((x_train.flatten(), y_train.flatten()), axis=-1)
y_train_true = z_train.flatten()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train_true, dtype=torch.float32).unsqueeze(1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate predictions for plotting
X_plot = np.stack((x_plot.flatten(), y_plot.flatten()), axis=-1)
X_plot_tensor = torch.tensor(X_plot, dtype=torch.float32)
z_pred = model(X_plot_tensor).detach().numpy().reshape(*z_plot.shape)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

surface1 = go.Surface(z=z_plot, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface1, row=1, col=1)

surface2 = go.Surface(z=z_pred, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface2, row=1, col=2)

fig.update_layout(title='3D Plot of f(x, y) = x * y and Neural Network Approximation',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  scene2=dict(xaxis_title='x',
                              yaxis_title='y',
                              zaxis_title='z_pred'),
                  width=1600, height=800)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim

def f(x, y):
    return x * y

# Generate a grid of values for plotting
x_plot = np.linspace(-1000, 1000, 1000)
y_plot = np.linspace(-1000, 1000, 1000)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
z_plot = f(x_plot, y_plot)

# Generate a grid of values for training
x_train = np.linspace(-10, 10, 100)
y_train = np.linspace(-10, 10, 100)
x_train, y_train = np.meshgrid(x_train, y_train)
z_train = f(x_train, y_train)

# Prepare data for training
X_train = np.stack((x_train.flatten(), y_train.flatten()), axis=-1)
y_train_true = z_train.flatten()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train_true, dtype=torch.float32).unsqueeze(1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 1024)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate predictions for plotting
X_plot = np.stack((x_plot.flatten(), y_plot.flatten()), axis=-1)
X_plot_tensor = torch.tensor(X_plot, dtype=torch.float32)
z_pred = model(X_plot_tensor).detach().numpy().reshape(*z_plot.shape)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

surface1 = go.Surface(z=z_plot, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface1, row=1, col=1)

surface2 = go.Surface(z=z_pred, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface2, row=1, col=2)

fig.update_layout(title='3D Plot of f(x, y) = x * y and Neural Network Approximation',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  scene2=dict(xaxis_title='x',
                              yaxis_title='y',
                              zaxis_title='z_pred'),
                  width=1600, height=800)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim

def f(x, y):
    return x * y

# Generate a grid of values for plotting
x_plot = np.linspace(-1000, 1000, 1000)
y_plot = np.linspace(-1000, 1000, 1000)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
z_plot = f(x_plot, y_plot)

# Generate a grid of values for training
x_train = np.linspace(-10, 10, 100)
y_train = np.linspace(-10, 10, 100)
x_train, y_train = np.meshgrid(x_train, y_train)
z_train = f(x_train, y_train)

# Prepare data for training
X_train = np.stack((x_train.flatten(), y_train.flatten()), axis=-1)
y_train_true = z_train.flatten()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train_true, dtype=torch.float32).unsqueeze(1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate predictions for plotting
X_plot = np.stack((x_plot.flatten(), y_plot.flatten()), axis=-1)
X_plot_tensor = torch.tensor(X_plot, dtype=torch.float32)
z_pred = model(X_plot_tensor).detach().numpy().reshape(*z_plot.shape)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

surface1 = go.Surface(z=z_plot, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface1, row=1, col=1)

surface2 = go.Surface(z=z_pred, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface2, row=1, col=2)

fig.update_layout(title='3D Plot of f(x, y) = x * y and Neural Network Approximation',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  scene2=dict(xaxis_title='x',
                              yaxis_title='y',
                              zaxis_title='z_pred'),
                  width=1600, height=800)

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.optim as optim

def f(x, y):
    return x * y

# Generate a grid of values for plotting
x_plot = np.linspace(-1000, 1000, 1000)
y_plot = np.linspace(-1000, 1000, 1000)
x_plot, y_plot = np.meshgrid(x_plot, y_plot)
z_plot = f(x_plot, y_plot)

# Generate a grid of values for training
x_train = np.linspace(-10, 10, 100)
y_train = np.linspace(-10, 10, 100)
x_train, y_train = np.meshgrid(x_train, y_train)
z_train = f(x_train, y_train)

# Prepare data for training
X_train = np.stack((x_train.flatten(), y_train.flatten()), axis=-1)
y_train_true = z_train.flatten()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train_true, dtype=torch.float32).unsqueeze(1)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)
        

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = self.fc4(x)
        return x

model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate predictions for plotting
X_plot = np.stack((x_plot.flatten(), y_plot.flatten()), axis=-1)
X_plot_tensor = torch.tensor(X_plot, dtype=torch.float32)
z_pred = model(X_plot_tensor).detach().numpy().reshape(*z_plot.shape)

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

surface1 = go.Surface(z=z_plot, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface1, row=1, col=1)

surface2 = go.Surface(z=z_pred, x=x_plot, y=y_plot, showscale=False)
fig.add_trace(surface2, row=1, col=2)

fig.update_layout(title='3D Plot of f(x, y) = x * y and Neural Network Approximation',
                  scene=dict(xaxis_title='x',
                             yaxis_title='y',
                             zaxis_title='z = x * y'),
                  scene2=dict(xaxis_title='x',
                              yaxis_title='y',
                              zaxis_title='z_pred'),
                  width=1600, height=800)

fig.show()
